In [1]:
from verl.utils import hf_tokenizer, hf_processor

In [2]:
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"
processor = hf_processor(model_name)
tokenizer = hf_tokenizer(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
tokenizer.special_tokens_map.items()

dict_items([('eos_token', '<|im_end|>'), ('pad_token', '<|endoftext|>'), ('additional_special_tokens', ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>'])])

In [26]:
ss="<|box_start|>I love you<|box_end|>"
ids=tokenizer(ss, add_special_tokens=False)
print(ids)
import re
ss=re.sub(r'<\|box_start\|>', '', ss)
ss=re.sub(r'<\|box_end\|>', '', ss)
print(ss)
ids=tokenizer(ss, add_special_tokens=False)
print(ids)

{'input_ids': [151648, 40, 2948, 498, 151649], 'attention_mask': [1, 1, 1, 1, 1]}
I love you
{'input_ids': [40, 2948, 498], 'attention_mask': [1, 1, 1]}


In [ ]:
import verl.utils.torch_functional as verl_F
prompt_with_chat_template="<|box_start|>I love you<|box_end|>"
rst=verl_F.tokenize_and_postprocess_data(prompt=prompt_with_chat_template,
                                        tokenizer=tokenizer,
                                        max_length=1024,
                                        pad_token_id=tokenizer.pad_token_id,
                                        left_pad=True,
                                        truncation="error")
print(rst)


(tensor([[151643, 151643, 151643,  ...,   2948,    498, 151649]]), tensor([[0, 0, 0,  ..., 1, 1, 1]]))


In [30]:
ss="<image1asda1231123124124asdas>asdasd wqdfqfvqw <image2>asdasda asdasd"
ss=re.sub(r'<image([^>]+)>', '<image>', ss)
print(ss)

<image>asdasd wqdfqfvqw <image>asdasda asdasd


In [ ]:
ids=tokenizer.convert_tokens_to_ids(["<image>"])

{'input_ids': [151648, 358, 2948, 498, 220, 151649], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [5]:
image1_path="/projects/p32476/projects/vagen/verl-vlm/vagen/mllm_agent/image.png"
image2_path="/projects/p32476/projects/vagen/verl-vlm/vagen/mllm_agent/image2.png"

In [10]:
import numpy as np
from PIL import Image

def load_image_for_processing(image_path):
    """
    Load an image from path in a format compatible with the process_image function.
    
    Args:
        image_path (str): Path to the image file
        
    Returns:
        dict: Dictionary with 'bytes' containing the image data
        OR
        PIL.Image: PIL Image object
    """
    try:
        # Open the image file
        with open(image_path, 'rb') as f:
            image_bytes = f.read()
        
        # Create a dictionary with 'bytes' key
        image_dict = {'bytes': image_bytes}
        
        # You can also return the PIL Image directly if preferred
        # image = Image.open(BytesIO(image_bytes))
        # return image
        
        print(f"Successfully loaded image from {image_path}")
        return image_dict
    
    except Exception as e:
        print(f"Error loading image: {str(e)}")
        return None

image1 = load_image_for_processing(image1_path)
image2 = load_image_for_processing(image2_path)

Successfully loaded image from /projects/p32476/projects/vagen/verl-vlm/vagen/mllm_agent/image.png
Successfully loaded image from /projects/p32476/projects/vagen/verl-vlm/vagen/mllm_agent/image2.png


In [ ]:
images = [image1, image2,image1]

In [37]:
chat=[]
chat.append({"role": "user", "content": "<image>What is the difference between two images car?<image>"})
chat.append({"role": "assistant", "content": "<think>"})
from verl.utils.dataset.rl_dataset import process_image
images=[process_image(image) for image in images]

In [38]:
new_images=processor.image_processor(images, return_tensors='pt')

In [ ]:
for k,v in new_images.items():
    print(k, v.new_imagesshape)

pixel_values torch.Size([13788, 1176])
image_grid_thw torch.Size([5, 3])


In [42]:
new_images["pixel_values"]

tensor([[-1.1061, -1.0477, -0.9602,  ..., -0.8688, -0.9256, -0.9541],
        [-0.9310, -0.9456, -0.9456,  ..., -0.9967, -1.0678, -1.1247],
        [-0.9748, -0.9748, -0.9602,  ..., -0.8830, -0.8972, -0.9399],
        ...,
        [-0.0405, -0.0696, -0.0988,  ..., -0.1293, -0.1009, -0.1009],
        [-0.0696, -0.0696, -0.0259,  ..., -0.2004, -0.2004, -0.2289],
        [-0.0113, -0.0842, -0.0988,  ..., -0.2289, -0.1862, -0.1009]])

In [18]:
prompt_with_chat_template = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False)

In [19]:
prompt_with_chat_template

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<image>What is the difference between two images car?<image><|im_end|>\n<|im_start|>assistant\n<think><|im_end|>\n<|im_start|>assistant\n'

In [ ]:
processor.image_processor.merge_size

2

In [30]:
image_grid_thw[0].prod()

tensor(4752)

In [31]:
image_grid_thw

tensor([[ 1, 54, 88],
        [ 1, 34, 42]])

In [32]:
new_images['image_grid_thw']

tensor([[ 1, 54, 88],
        [ 1, 34, 42],
        [ 1, 34, 42]])

In [1]:
from collections import defaultdict
a=defaultdict(dict)
a[1][2]=3
a[1][3]=4
print(a)

defaultdict(<class 'dict'>, {1: {2: 3, 3: 4}})


In [ ]:
image_grid_thw=new_images['image_grid_thw']
if image_grid_thw is not None:
    merge_length = processor.image_processor.merge_size**2
    index = 0
    while '<image>' in prompt_with_chat_template:
        prompt_with_chat_template = prompt_with_chat_template.replace(
            '<image>',
            '<|vision_start|>' + '<|placeholder|>' * (image_grid_thw[index].prod() // merge_length) +
            '<|vision_end|>',
            1,
        )
        index += 1
    
    prompt_with_chat_template = prompt_with_chat_template.replace('<|placeholder|>',
                                                                    processor.image_token)

In [28]:
processor.image_token

'<|image_pad|>'

In [22]:
prompt_with_chat_template

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_